In [53]:
# !pip install pdfminer pypdf2 tqdm openpyxl pandas gspread df2gspread psycopg2

In [54]:
# !exa -l data

In [65]:
# !exa data -T

In [68]:
import io
import re
import os
import sys
from collections import defaultdict

import pandas as pd
from tqdm.notebook import tqdm

from utils.dataframe import gdrive_sheets, df2gsheet, gsheet
from utils.pdf_processing import (get_all_data, 
                                  get_text_object_details, 
                                  pdfminer_to_text, pypdf2txt)
from utils.text_processing import find

<Spreadsheet 'college_catalogs_data' id:12pYVv4uYV5HGrNH4_tkFSZqugJxO4kOzLFLsXwhamNI>


In [33]:
# done = False
# for directory,_,filenames in os.walk('data'):
#     if directory.lstrip('data/') in parse_only:
#         for filename in filenames:
#             data = pdf_to_text(directory+'/'+filename)
#             done = True
#             break
#     if done: break

# Arkansas

In [152]:
arkansas_sample = pdf_to_text('data/Arkansas/2019-2020.pdf')

In [153]:
pd.DataFrame(re.findall(r"(\n\n?[A-Z]{4}\s[0-9]{3,4}\.[\(\w\s.\n,\.:-]*)",arkansas_sample))

,0
0,\nENGL 0002.\n\n
1,\nMATH 1203.\n\nStudents will be required to r...
2,\nCOMM 1313. Both the written and oral (a five...
3,\nPLSC 2003.\n\n98 University Core\n\n...
4,\nANTH 1023. At least 9 hours must be in cours...
...,...
5026,\n\nWLIT 5623. The Bible as Literature. 3 Hour...
5027,\n\nWLIT 5993. African Literature. 3 Hours.\n(...
5028,\n\nWLIT 6703. Psychoanalysis and Culture. 3 H...
5029,"\n\nWLIT 6713. Literature of Spain, 711-1615 C..."


# Bowdowin

In [110]:
# pdf2txt doesnt work
bowdowin_sample = pdf_to_text("data/Bowdoin/Bowdoin_2019-2020.pdf")
bs = bowdowin_sample

In [346]:
# find = "AFRS 1012"
# bs[bs.index(find)-10:bs.index(find)+2000]
bowdoin_dfs = []
for file in tqdm(os.listdir('data/Bowdoin')):
    text = pdf_to_text(f"data/Bowdoin/{file}")
    if '202' in file:
        data = [{'id':r[1],'title':r[3],'sub':r[4],'desc':r[6],'num':r[5],'catalog':file} 
            for r in re.findall(
                r"("
                    r"\n\n([A-Z]{4}(\s|\\xa0)[0-9]{4})"  # course id
                    r"(.*?)\n"                           # course title
                    r"(.*?)"                             # course subtitle
                    r"\s([0-9]{1,3})\."                  # course enrolment limit
                    r"((?:(?![\n]{1}[A-Z]{4}).)*)"       # course description 
                                                          # UNTIL next course id 
                                                          # but not including it
                r")",
                text,
                re.DOTALL)
        ]
    else: #pre 2020
        data = [{'id':r[1],'title':r[3],'sub':None,'desc':r[4],'num':None,'catalog':file} 
                for r in re.findall(
                r"("
                    r"\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)"  # course id
                    r"(.*?)\n"                           # course title
                    r"((?:(?!\n([0-9]{2,3}[a-z]{0,1})(\.\s|\s-)).)*)"  # UNTIL next course id but not including
                r")",
                text,
                re.DOTALL)
        ]
        
    bowdoin_dfs.append(pd.DataFrame(data))

In [365]:
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


In [368]:
df_bowdoin = pd.concat(bowdoin_dfs)
df_bowdoin['title'] = df_bowdoin['title'].str.replace('\n','')
# df_bowdoin['title'] = df_bowdoin['title'].str.replace(r'\x','')
df_bowdoin['desc'] = df_bowdoin['desc'].str.replace('\n','')
# df_bowdoin['desc'] = df_bowdoin['desc'].str.replace(r'\x','')
df_bowdoin = df_bowdoin.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
with pd.ExcelWriter('data/bowdoin.xlsx') as writer: 
    df_bowdoin.to_excel(writer, sheet_name='data')
df_bowdoin

,id,title,sub,desc,num,catalog
0,10b,Racism. Fall 2010. Spring 2011. Roy Partridge....,None,12c. Affirmative Action and United States Soci...,None,Bowdoin_2010-2011.pdf
1,13c,From Montezuma to Bin Laden: Globalization and...,None,Gordon. (Same as History 16.),None,Bowdoin_2010-2011.pdf
2,16c,Fictions of Freedom. Fall 2010. Tess Chakkalak...,None,[25c. The Civil War in Film. (Same as History ...,None,Bowdoin_2010-2011.pdf
3,101c,Introduction to Africana Studies. Every fall. ...,None,Focuses on major humanities and social science...,None,Bowdoin_2010-2011.pdf
4,107c,ESD. Introduction to the Black Novel in the U...,None,Chakkalakal.Introduces students to the literar...,None,Bowdoin_2010-2011.pdf
...,...,...,...,...,...,...
840,320c,Theater Styles. Every third year. Fall 2009. D...,None,An advanced acting class that explores issues ...,None,Bowdoin_2009-2010.pdf
841,321c,Comedy in Performance. Every third year. Fall ...,None,"Looks at several facets of comedy on stage, fr...",None,Bowdoin_2009-2010.pdf
842,322c,Ensemble: Theater and Dance Collaborative...,None,DePaRtMent.Experienced theater and dance stude...,None,Bowdoin_2009-2010.pdf
843,323c,Acting Shakespeare. Spring 2010. abiGail killeen.,None,An advanced-level acting course dedicated to t...,None,Bowdoin_2009-2010.pdf


# Ohio State

In [9]:
ohio_sample = pypdf2txt("data/Ohio State/course_catalog_2010_2011.pdf")

In [154]:
ohio_dfs = []
for file in tqdm(os.listdir('data/Ohio State')):
    # better to use pdf2txt, otherwise the number to the left of letter U
    # sometimes flotes away from the course it is assigned to
    ohs2 = pdf2txt(f"data/Ohio State/{file}")

    data = [{'id':r[1],'title':r[6],'desc':r[7],'num':r[8],'catalog':file} 
     for r 
     in re.findall(
         r"("
            r"\n("                           # course ID
                r"([0-9]{4}\.[0-9A-Z]{3})"     # e.g. 2367.07S
                r"|([0-9]{4}\.[0-9]{2})"       # e.g. 2367.04
                r"|([0-9]{4}[H])"              # e.g. 4780H
                r"|([0-9]{4})"                 # e.g. 3080
            r")"
           r"\n([A-Za-z\s]*)"                # course title
           r"\nU\n"                          # letter U
           r"(.*?)"                          # course desrciption
           r"\n([0-9]{1,2})"                 # number to the right of letter U
         r")",
         ohs2, 
         re.DOTALL) 
    ]
    ohio_dfs.append(pd.DataFrame(data))    

In [155]:
df_ohio = pd.concat(ohio_dfs)
df_ohio['title'] = df_ohio['title'].str.replace('\n','')
df_ohio['desc'] = df_ohio['desc'].str.replace('\n','')
with pd.ExcelWriter('data/ohio.xlsx') as writer: 
    df_ohio.to_excel(writer, sheet_name='data')
df_ohio

,id,title,desc,num,catalog
0,2000,Foundations of Accounting,A survey of accounting for non-business studen...,3,course_catalog_2015_2016.pdf
1,2200,Introduction to Accounting I,Preparation and use of accounting reports for ...,3,course_catalog_2015_2016.pdf
2,2200H,Introduction to Accounting I,Structure and use of financial reports; role o...,3,course_catalog_2015_2016.pdf
3,2300,Introduction to Accounting II,The uses of accounting reports by managers to ...,20,course_catalog_2015_2016.pdf
4,2300H,Introduction to Accounting II,Use of accounting information in management co...,3,course_catalog_2015_2016.pdf
...,...,...,...,...,...
3899,4721,Studies in Yiddish Literature,"Advanced study of specific literary periods, f...",3,Course Bulletin 2019-2020.pdf
3900,4998,Undergraduate Research,Investigation of problems in various fields of...,1,Course Bulletin 2019-2020.pdf
3901,4998H,Honors Research,Investigation of problems in various fields of...,3,Course Bulletin 2019-2020.pdf
3902,4999,Undergraduate Research Thesis,Research for undergraduate thesis.Prereq: Perm...,1,Course Bulletin 2019-2020.pdf


### Departments

In [49]:
departments_mappings = []
for file in tqdm(os.listdir('data/Ohio State')):
    mapping = get_departments(f"data/Ohio State/{file}")
    cleaned_mapping = dict()
    department_to_courses_ids_mapping = dict()
    
    for k,v in mapping.items():
        if len(v) > 50:
            cleaned_mapping[k] = v
            
    
    for k,v in cleaned_mapping.items():
        for r in re.findall(r'\n [0-9]{3,4}[\.0-9]{0,3}H?\n', v):
            department_to_courses_ids_mapping[r.strip()] = k
            
    try:
        departments_mappings.append(pd.Series(department_to_courses_ids_mapping, name=file))
    except:
        pass

<ipython-input-49-89da535d95b6>:17: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  departments_mappings.append(pd.Series(department_to_courses_ids_mapping, name=file))


KeyboardInterrupt: 